In [1]:
# Load the required libraries
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from PIL import Image as im
import pickle
import cv2
from tqdm import tqdm
from scipy.ndimage import rotate
import time
import joblib
from sklearn.metrics import accuracy_score,f1_score,classification_report
import os
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC
from sklearn.model_selection import RandomizedSearchCV
from sklearn.ensemble import RandomForestClassifier

In [2]:
labels = [ 'Scheherazade New' , 'Marhey' , 'Lemonada' , 'IBM Plex Sans Arabic']
image_size = 600
def show_images(images,titles=None):
    #This function is used to show image(s) with titles by sending an array of images and an array of associated titles.
    # images[0] will be drawn with the title titles[0] if exists
    # You aren't required to understand this function, use it as-is.
    n_ims = len(images)
    if titles is None: titles = ['(%d)' % i for i in range(1,n_ims + 1)]
    fig = plt.figure()
    n = 1
    for image,title in zip(images,titles):
        a = fig.add_subplot(1,n_ims,n)
        if image.ndim == 2: 
            plt.gray()
        plt.imshow(image)
        a.set_title(title)
        plt.axis('off')
        n += 1
    fig.set_size_inches(np.array(fig.get_size_inches()) * n_ims)
    plt.show() 

In [3]:
# Load the images from fonts-dataset folder
def load_images():
    # Load the images from the fonts-dataset folder
    images_train = []
    labels_train = []
    filenames = []
    empty_images_filenames = ["360.jpeg","627.jpeg","853.jpeg"] 
    # Use tqdm to show a progress bar
    for i in tqdm(labels):
        for filename in os.listdir(f'fonts-dataset/{i}'):
            img = cv2.imread(f'fonts-dataset/{i}/{filename}', cv2.IMREAD_GRAYSCALE)
            # img = cv2.resize(img, (image_size, image_size))
            if i == "Lemonada" and filename in empty_images_filenames:
                print(filename)
                print("empty image")
                continue
            images_train.append(img)
            labels_train.append(i)
            filenames.append(filename)
    return images_train, labels_train,filenames



In [4]:
# Load the images
X_train, y_train_org, filenames = load_images()
# Change the y_train to numbers
y_train_org = [labels.index(i) for i in y_train_org]

 50%|█████     | 2/4 [00:14<00:14,  7.45s/it]

360.jpeg
empty image
627.jpeg
empty image
853.jpeg
empty image


100%|██████████| 4/4 [00:28<00:00,  7.20s/it]


In [5]:
len(X_train), len(y_train_org), len(filenames)

(3997, 3997, 3997)

In [6]:
def find_score(arr, angle):
    """
    Find the score of the skew angle to be used in deskewing the image
    
    Args:
    arr: the image array
    angle: the angle to rotate the image by
    
    Returns:
    hist: the histogram of the image
    score: the score of the skew angle
    """
    
    # mode{‘reflect’, ‘grid-mirror’, ‘constant’, ‘grid-constant’, ‘nearest’, ‘mirror’, ‘grid-wrap’, ‘wrap’}
    data = rotate(arr, angle, reshape=False, order=0, mode='constant', cval=0, prefilter=False)
    hist = np.sum(data, axis=1)
    score = np.sum((hist[1:] - hist[:-1]) ** 2)
    return hist, score

def rotate_image(image, angle):
    """
    Rotates an image by a given angle and fills the remaining pixels with white color.

    Args:
        image: A NumPy array representing the input image.
        angle: The rotation angle in degrees.

    Returns:
        A new NumPy array representing the rotated image.
    """
    # Get image height and width
    height, width = image.shape[:2]

    # Compute the rotation matrix
    rotation_matrix = cv2.getRotationMatrix2D((width / 2, height / 2), angle, 1)

    # Perform the rotation and fill the remaining pixels with white color
    rotated_image = cv2.warpAffine(image, rotation_matrix, (width, height), flags=cv2.INTER_LINEAR, borderMode=cv2.BORDER_CONSTANT, borderValue=(1, 1, 1))

    return rotated_image

def deskew(binary_img):
    """
    Deskew the image
    
    Args:
    binary_img: the binary image
    
    Returns:
    pix: the deskewed image
    """
    bin_img = (binary_img // 255.0)
    # angles to check for skew angle = 45 degrees and 90 degrees and 180
    angles = np.array ([0 , 45 , 90 , 135 , 180 , 225 , 270 , 315])
    scores = []
    for angle in angles:
        hist, score = find_score(bin_img, angle)
        scores.append(score)

    best_score = max(scores)
    best_angle = angles[scores.index(best_score)]
    # print('Best angle: {}'.format(best_angle))

    # correct skew
    # data = rotate(bin_img, best_angle, reshape=False, order=0)
    data = rotate_image(bin_img, best_angle)
    img = im.fromarray((255 * data).astype("uint8"))

    pix = np.array(img)
    return pix

In [7]:
def preprocess(img):
    """
    Preprocess the image
    
    Args:
    img: the image
    
    Returns:
    img: the preprocessed image
    """
    sharpen_kernel = np.array([[0,-1, 0], [-1,5,-1], [0,-1,0]])
    img = cv2.medianBlur(img, 3) # To remove Salt and Pepper noise
    img = cv2.filter2D(img, -1, sharpen_kernel)  # Sharpen the image
    img = cv2.threshold(img, 0, 255, cv2.THRESH_BINARY + cv2.THRESH_OTSU)[1] # Convert the image to binary
    deskewed_img = deskew(img) # Deskew the image
    final_img = cv2.bitwise_not(deskewed_img) if np.mean(deskewed_img) > 127 else deskewed_img # Invert the image if the mean is less than 127 
    return final_img

In [8]:
# # Preprocess the images
# X_train_preprocess = [preprocess(i) for i in tqdm(X_train)]
    
# # Dump the preprocessed images to a file
# with open('preprocessed_images.pkl', 'wb') as f:
#     pickle.dump(X_train_preprocess, f)

In [9]:
# Load the preprocessed images
with open('preprocessed_images.pkl', 'rb') as f:
    X_train_preprocess = pickle.load(f)

In [10]:
import numpy as np
import cv2 as cv


def save_image(img, folder, title):
    cv.imwrite(f'./{folder}/{title}.png', img)

def projection(gray_img, axis:str='horizontal'):
    """ Compute the horizontal or the vertical projection of a gray image """

    if axis == 'horizontal':
        projection_bins = np.sum(gray_img, 1).astype('int32')
    elif axis == 'vertical':
        projection_bins = np.sum(gray_img, 0).astype('int32')

    return projection_bins

In [11]:
# def preprocess(image):

#     # Maybe we end up using only gray level image.
#     # gray_img = cv.bitwise_not(image) # Invert the image
#     binary_img = cv.threshold(image, 0, 255, cv.THRESH_BINARY + cv.THRESH_OTSU)[1]
#     deskewed_img = deskew(binary_img)
#     return deskewed_img


def projection_segmentation(clean_img, axis, cut=15, min_width=20, min_height=30):
    """Segment the image based on the projection profile

    Args:
        clean_img : Preprocessed image
        axis (str): 'horizontal' or 'vertical'
        cut (int, optional): Gap between the segments. Defaults to 3.
        min_width (int, optional): Width of the segment. Defaults to 5.
        min_height (int, optional): Height of the segment. Defaults to 5.

    Returns:
        _type_: _description_
    """
    segments = []
    start = -1
    cnt = 0

    projection_bins = projection(clean_img, axis)
    for idx, projection_bin in enumerate(projection_bins):

        if projection_bin != 0:
            cnt = 0
        if projection_bin != 0 and start == -1:
            start = idx
        if projection_bin == 0 and start != -1:
            cnt += 1
            if cnt >= cut:
                if axis == 'horizontal':
                    # Line segmentation
                    segment = clean_img[max(start-1, 0):idx, :]
                    # if segment.shape[0] >= min_height:                    
                    segments.append(segment)
                elif axis == 'vertical':
                    # Word segmentation
                    segment = clean_img[:, max(start-1, 0):idx]
                    # if segment.shape[1] >= min_width:
                    segments.append(segment)
                cnt = 0
                start = -1
    
    return segments


# Line Segmentation
#----------------------------------------------------------------------------------------
def line_horizontal_projection(image, cut=3):

    # Segmentation    
    lines = projection_segmentation(image, axis='horizontal', cut=cut)
    return lines


# Word Segmentation
#----------------------------------------------------------------------------------------
def word_vertical_projection(line_image, cut=3):
    
    line_words = projection_segmentation(line_image, axis='vertical', cut=cut)
    line_words.reverse()
    return line_words


def extract_words(img, visual=0):

    lines = line_horizontal_projection(img)
    words = []
    
    for idx, line in enumerate(lines):
        
        if visual:
            # Check for the size of the line to be greater than 30
            # if line.shape[0] > 30:
            save_image(line, 'lines', f'line{idx}')

        line_words = word_vertical_projection(line)
        for w in line_words:
            # if len(words) == 585:
            #     print(idx)
            words.append((w, line))
        # words.extend(line_words)

    # breakpoint()
    if visual:
        for idx, word in enumerate(words):
            # check for the size of the word to be greater than 30
            # print (word[0].shape)
            # if word[0].shape[0] < 100 and word[0].shape[1] > 20 :
            save_image(word[0], 'words', f'word{idx}')
    return words

# # Try to extract the words from the preprocessed images
# x = 31
# p = preprocess(X_train[x])

# # show_images([X_train[999],p])

# # Extract the words from the preprocessed images
# words = extract_words(p, visual=1)
# show_images([X_train[x], p])



In [12]:
# Resize the images to 600x600
X_train_resized = []
for i in tqdm(X_train_preprocess):
    img = cv2.resize(i, (image_size, image_size))
    X_train_resized.append(img)
X_train_resized = np.array(X_train_resized)
    

  0%|          | 0/4000 [00:00<?, ?it/s]

100%|██████████| 4000/4000 [00:02<00:00, 1538.64it/s]


In [13]:
# Remove from X_train_resized with index = [2291, 2587, 2838]
X_train_resized = np.delete(X_train_resized, [2291, 2587, 2838], 0)

In [14]:
# Apply hog for the images
from skimage.feature import hog

X_train_hog = []
for i in tqdm(X_train_resized):
    X_train_hog.append(hog(i, orientations= 16, pixels_per_cell=(32, 32), cells_per_block=(4, 4), block_norm='L2-Hys'))
    

X_train_hog = np.array(X_train_hog)
print(X_train_hog.shape)
# # Try hog on one image
# hog_image = hog(X_train_resized[0], orientations= 16, pixels_per_cell=(32, 32), cells_per_block=(4,4), block_norm='L2-Hys')
# print(hog_image.shape)

100%|██████████| 3997/3997 [02:40<00:00, 24.93it/s]


(3997, 57600)


In [15]:
# Apply SIFT for the images
sift = cv2.SIFT_create()

X_train_sift = []
for i in tqdm(X_train_resized):
    kp, des = sift.detectAndCompute(i, None)
    if des is None:
        # Add a row of zeros to the SIFT descriptors
        des = np.zeros((1, 128))
    des = des.flatten()
    X_train_sift.append(des)


  0%|          | 0/3997 [00:00<?, ?it/s]

100%|██████████| 3997/3997 [03:39<00:00, 18.18it/s]


In [16]:
# Get the maximum keypoint length of the SIFT descriptors
max_kp = max(len(kp)/128 for kp in X_train_sift)

# Get the average keypoint length of the SIFT descriptors
avg_kp = np.mean([len(kp)/128 for kp in X_train_sift])

# Get the minimum keypoint length of the SIFT descriptors
min_kp = min(len(kp)/128 for kp in X_train_sift)

# Print the maximum keypoint length
print(max_kp)

# Print the average keypoint length
print(avg_kp)

# Print the minimum keypoint length
print(min_kp)


11506.0
1265.8418814110582
23.0


In [17]:
# Get the index of the image with keypoint <= 10
idx = [i for i in range(len(X_train_sift)) if len(X_train_sift[i])/128 <= 50]
# Print idx 
print(idx)

# Found images which has no text 

[1783, 3780]


In [18]:
# Pad the SIFT descriptors to the maximum length
fixed_len = 128 * 350  # Adjust this as needed

# Create a generator that yields each padded descriptor on-the-fly
padded_descriptors = (np.pad(des, (0, max(0, fixed_len - des.shape[0])))[:fixed_len] for des in X_train_sift)

# Convert the generator to a numpy array
X_train_sift_np = np.array(list(padded_descriptors))

# Print the shape of the numpy array
print(X_train_sift_np.shape)

(3997, 44800)


In [19]:
# # Concatenate the hog and sift features
X_train_features = np.concatenate((X_train_hog, X_train_sift_np), axis=1)
print(X_train_features.shape)


(3997, 102400)


In [20]:
# Split the data into training and testing
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X_train_features, y_train_org, test_size=0.15, random_state=42, stratify=y_train_org)

In [23]:
del X_train_features

In [72]:
# Make a pipeline for the model which consist of StandardScaler , PCA and the model
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA

# Create a pipeline
pipe = Pipeline([
    ('scaler', StandardScaler()),
    ('pca', PCA(n_components=0.99)),
])

# Fit the pipeline
pipe.fit(X_train)

# Dump the pipeline to a file
joblib.dump(pipe, 'pipeline.pkl')

['pipeline.pkl']

In [21]:
# Load the pipeline
pipe = joblib.load('pipeline.pkl')

In [24]:
# Transform the data
X_train_transformed = pipe.transform(X_train)
X_test_transformed = pipe.transform(X_test)

# Print the shape of the transformed data
print(X_train_transformed.shape)

(3397, 3165)


In [25]:
# Build a shallow neural network model of 1 hidden layer with 256 neurons and relu activation function
from sklearn.neural_network import MLPClassifier
from sklearn.metrics import accuracy_score

# Create a model of 1 hidden layer with 256 neurons and relu activation function and adam solver and softmax output layer
model = MLPClassifier(hidden_layer_sizes=(256,), activation='relu', solver='adam', verbose=True)

# Fit the model
model.fit(X_train_transformed, y_train)

# Predict the training data
y_train_pred = model.predict(X_train_transformed)

# Predict the testing data
y_test_pred = model.predict(X_test_transformed)

# Print the accuracy of the model
print('Train accuracy: ', accuracy_score(y_train, y_train_pred)*100)

print('Test accuracy: ', accuracy_score(y_test, y_test_pred)*100)

# Print the classification report
print('Train classification report: ', classification_report(y_train, y_train_pred, target_names=labels))

print('Test classification report: ', classification_report(y_test, y_test_pred, target_names=labels))

# Accuracy: 92.75%

Iteration 1, loss = 4.00282194
Iteration 2, loss = 0.05200491
Iteration 3, loss = 0.00889531
Iteration 4, loss = 0.00077065
Iteration 5, loss = 0.00052960
Iteration 6, loss = 0.00042028
Iteration 7, loss = 0.00037226
Iteration 8, loss = 0.00034184
Iteration 9, loss = 0.00031879
Iteration 10, loss = 0.00030113
Iteration 11, loss = 0.00028635
Iteration 12, loss = 0.00027438
Iteration 13, loss = 0.00026326
Iteration 14, loss = 0.00025406
Iteration 15, loss = 0.00024584
Iteration 16, loss = 0.00023836
Iteration 17, loss = 0.00023136
Training loss did not improve more than tol=0.000100 for 10 consecutive epochs. Stopping.
Train accuracy:  100.0
Test accuracy:  92.0
Train classification report:                        precision    recall  f1-score   support

    Scheherazade New       1.00      1.00      1.00       850
              Marhey       1.00      1.00      1.00       850
            Lemonada       1.00      1.00      1.00       847
IBM Plex Sans Arabic       1.00      1.00      1.00 

In [151]:
# lin_model = LogisticRegression()

# # Define the hyperparameters
# param_grid = {
#     'C': np.logspace(-4, 4, 30),
#     'penalty': ['l2'],
#     'solver': ['liblinear', 'saga', 'lbfgs'],
#     'warm_start': [True, False]
# }

# # Initialize the RandomizedSearchCV
# random_search = RandomizedSearchCV(lin_model, param_distributions=param_grid, n_iter=100, cv=5, verbose=2, random_state=42, n_jobs=-1)

# # Fit the model
# random_search.fit(X_train_transformed, y_train)
# # cv_results_df = pd.DataFrame(clf_searched.cv_results_)[relevant_columns].round(decimals=3).sort_values(by='rank_test_score')
# # cv_results_df.head(10)
# relevant_columns = ['param_C', 'param_penalty', 'param_solver', 'param_warm_start', 'mean_test_score', 'std_test_score', 'rank_test_score']
# cv_results_df = pd.DataFrame(random_search.cv_results_)[relevant_columns].round(decimals=3).sort_values(by='rank_test_score')
# cv_results_df.head(10)

# # Print the best parameters
# print(random_search.best_params_)

In [76]:
# Initialize the model and fit the data

# Initialize the model
# {'warm_start': True, 'solver': 'saga', 'penalty': 'l2', 'C': 0.001}

# {'warm_start': True, 'solver': 'lbfgs', 'penalty': 'l2', 'C': 0.0006723357536499335}
model = LogisticRegression(warm_start=True, solver='saga', penalty='l2', C=0.8, random_state=42)
# model = LogisticRegression()

# Fit the model
model.fit(X_train_transformed, y_train)

# Predict the test data
y_pred = model.predict(X_test_transformed)

# Predict the train data
y_pred_train = model.predict(X_train_transformed)

# Print the training accuracy
print(f"Training Accuracy: {accuracy_score(y_train, y_pred_train)*100}")

# Print the testing accuracy
print(f"Testing Accuracy: {accuracy_score(y_test, y_pred)*100}")

# Print the f1 score of the training data
print(f"Training F1 Score: {f1_score(y_train, y_pred_train, average='weighted')}")

# Print the f1 score of the testing data
print (f"Testing F1 Score: {f1_score(y_test, y_pred, average='weighted')}")

# Print the classification report of the training data
print(classification_report(y_train, y_pred_train, target_names=labels))

# Print the classification report of the testing data
print(classification_report(y_test, y_pred, target_names=labels))

# Save the model
joblib.dump(model, 'logistic_model.pkl')

#ACCUARCY: 95.5%

Training Accuracy: 99.97056226081837
Testing Accuracy: 95.83333333333334
Training F1 Score: 0.999705622506323
Testing F1 Score: 0.9584696914637353
                      precision    recall  f1-score   support

    Scheherazade New       1.00      1.00      1.00       850
              Marhey       1.00      1.00      1.00       850
            Lemonada       1.00      1.00      1.00       847
IBM Plex Sans Arabic       1.00      1.00      1.00       850

            accuracy                           1.00      3397
           macro avg       1.00      1.00      1.00      3397
        weighted avg       1.00      1.00      1.00      3397

                      precision    recall  f1-score   support

    Scheherazade New       0.97      0.95      0.96       150
              Marhey       0.92      0.95      0.93       150
            Lemonada       0.99      0.99      0.99       150
IBM Plex Sans Arabic       0.95      0.94      0.95       150

            accuracy                      

C:\Users\ahmed\AppData\Roaming\Python\Python311\site-packages\sklearn\linear_model\_sag.py:350: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn(


['logistic_model.pkl']

In [77]:
# svm = SVC()

# param_dist = {
#     'C': np.logspace(-3, 3, 15), 
#     'kernel': ['poly', 'rbf'], 
#     'degree': [2, 3],
#     'gamma': ['scale', 'auto']
# }

# clf_searched = RandomizedSearchCV(svm, param_dist, n_iter=100, cv=5, random_state=42, n_jobs=-1, verbose=1)

# clf_searched.fit(X_train_transformed, y_train)
# relevant_columns = ['param_C', 'param_kernel', 'param_gamma', 'mean_test_score', 'std_test_score', 'rank_test_score']
# cv_results_df = pd.DataFrame(clf_searched.cv_results_)[relevant_columns].round(decimals=3).sort_values(by='rank_test_score')
# cv_results_df.head(10)

# Best parameters of the SVM
# {C : 7.196857 , kernel : 'rbf', gamma : 'scale'}

In [78]:
# Initialize the SVM model
svm = SVC(C=7.196857 , kernel='rbf', gamma='scale', random_state=42)

# Fit the SVM on the training data
svm.fit(X_train_transformed, y_train)

# Predict the labels of the test set
y_pred = svm.predict(X_test_transformed)
# Predict the labels of the training set
y_pred_train = svm.predict(X_train_transformed)

# Print the accuracy of the SVM model on the training set
print(f"Accuracy of SVM model on the training set: {accuracy_score(y_train, y_pred_train)*100}")

# Print the accuracy of the SVM model on the test set
print(f"Accuracy of SVM model: {accuracy_score(y_test, y_pred)*100}")

# Print the classification report of the SVM model
print(classification_report(y_train, y_pred_train, target_names=labels))

# Print the classification report of the SVM model
print(classification_report(y_test, y_pred, target_names=labels))

# Save the SVM model as h5 file
joblib.dump(svm, 'svm_model.pkl')

#ACCURACY: 91.25%

Accuracy of SVM model on the training set: 100.0
Accuracy of SVM model: 90.16666666666666
                      precision    recall  f1-score   support

    Scheherazade New       1.00      1.00      1.00       850
              Marhey       1.00      1.00      1.00       850
            Lemonada       1.00      1.00      1.00       847
IBM Plex Sans Arabic       1.00      1.00      1.00       850

            accuracy                           1.00      3397
           macro avg       1.00      1.00      1.00      3397
        weighted avg       1.00      1.00      1.00      3397

                      precision    recall  f1-score   support

    Scheherazade New       0.99      0.65      0.79       150
              Marhey       0.89      0.96      0.92       150
            Lemonada       1.00      1.00      1.00       150
IBM Plex Sans Arabic       0.79      0.99      0.88       150

            accuracy                           0.90       600
           macro avg       0.92      

['svm_model.pkl']

In [79]:
# # Random Forest Classifier

# random_forest_clf = RandomForestClassifier(random_state=42)

# # Define the hyperparameters distribution (use same ranges as before)
# param_dist = {
#     'n_estimators': np.arange(200, 500), 
#     'min_samples_split':  range(2,51),                    # 2 to 50
#     'min_samples_leaf': range(1,21),                      # 1 to 20
#     'max_depth': range(5, 51, 5),                              # 5 to 50 with step of 5
#     'min_impurity_decrease': np.linspace(0.0, 0.2, 20),                   # Decide a reasonable range here (with 20 values)
# }

# # Initialize RandomizedSearchCV
# clf_searched = RandomizedSearchCV(random_forest_clf, param_dist, n_iter=100, cv=5, random_state=42, n_jobs=-1, verbose=1)

# clf_searched.fit(X_train_transformed, y_train)
# relevant_columns = ['param_n_estimators', 'param_min_samples_split', 'param_min_samples_leaf', 'param_max_depth', 'param_min_impurity_decrease', 
#                     'mean_test_score', 'std_test_score', 'rank_test_score']
# cv_results_df = pd.DataFrame(clf_searched.cv_results_)[relevant_columns].round(decimals=3).sort_values(by='rank_test_score')
# cv_results_df.head(10)

# # Best parameters of the Random Forest Classifier
# # {'n_estimators': 371, 'min_samples_split': 48, 'min_samples_leaf': 4, 'min_impurity_decrease': 0.0, 'max_depth': 30}
# # Best score: 0.776

In [80]:
random_forest_clf = RandomForestClassifier(n_estimators=371, min_samples_split=48, min_samples_leaf=4, min_impurity_decrease=0., max_depth=30, random_state=42)

# Fit the Random Forest Classifier
random_forest_clf.fit(X_train_transformed, y_train)

# Predict the labels of the test set
y_pred = random_forest_clf.predict(X_test_transformed)

# Predict the labels of the training set
y_pred_train = random_forest_clf.predict(X_train_transformed)

# Print the accuracy of the Random Forest Classifier model on the training set
print(f"Accuracy of Random Forest Classifier model on the training set: {accuracy_score(y_train, y_pred_train)*100}")

# Print the accuracy of the Random Forest Classifier model on the test set
print(f"Accuracy of Random Forest Classifier model: {accuracy_score(y_test, y_pred)*100}")

# Print the classification report of the Random Forest Classifier model
print(classification_report(y_train, y_pred_train, target_names=labels))

# Print the classification report of the Random Forest Classifier model
print(classification_report(y_test, y_pred, target_names=labels))

# Save the Random Forest Classifier model as h5 file
joblib.dump(random_forest_clf, 'random_forest_clf.pkl')

#ACCURACY: 83.25%

Accuracy of Random Forest Classifier model on the training set: 99.85281130409184
Accuracy of Random Forest Classifier model: 78.33333333333333
                      precision    recall  f1-score   support

    Scheherazade New       1.00      1.00      1.00       850
              Marhey       1.00      1.00      1.00       850
            Lemonada       1.00      1.00      1.00       847
IBM Plex Sans Arabic       1.00      0.99      1.00       850

            accuracy                           1.00      3397
           macro avg       1.00      1.00      1.00      3397
        weighted avg       1.00      1.00      1.00      3397

                      precision    recall  f1-score   support

    Scheherazade New       0.88      0.61      0.72       150
              Marhey       0.60      0.93      0.73       150
            Lemonada       0.96      0.97      0.96       150
IBM Plex Sans Arabic       0.82      0.63      0.71       150

            accuracy                         

['random_forest_clf.pkl']

In [30]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, TensorDataset
from sklearn.base import BaseEstimator, ClassifierMixin
from sklearn.preprocessing import LabelBinarizer

class PyTorchClassifier(BaseEstimator, ClassifierMixin):
    def __init__(self, input_dim, hidden_dim1, hidden_dim2, output_dim , learning_rate=0.0002 , epoch=15):
        self.input_dim = input_dim
        self.hidden_dim1 = hidden_dim1
        self.hidden_dim2 = hidden_dim2
        self.output_dim = output_dim
        self.best_accuracy = -1  # Initialize with a value that will definitely be improved upon
        self.learning_rate = learning_rate
        self.epoch = epoch
        self.model = self.create_model()

    def create_model(self):
        model = nn.Sequential(
            nn.Linear(self.input_dim, self.hidden_dim1),
            nn.ReLU(),
            nn.Linear(self.hidden_dim1, self.hidden_dim2),
            nn.ReLU(),
            nn.Linear(self.hidden_dim2, self.output_dim),
        )
        return model
    
    def fit(self, X_train, X_val, y_train, y_val):
        lb = LabelBinarizer()
        y_train_one_hot = lb.fit_transform(y_train)
        y_val_one_hot = lb.fit_transform(y_val)

        X_train_tensor = torch.FloatTensor(X_train)
        y_train_tensor = torch.LongTensor(y_train_one_hot)

        X_val_tensor = torch.FloatTensor(X_val)
        y_val_tensor = torch.LongTensor(y_val_one_hot)
        
        # Create a dataset
        train_dataset = TensorDataset(X_train_tensor, y_train_tensor)
        test_dataset = TensorDataset(X_val_tensor, y_val_tensor)

        # Create a dataloader
        train_loader = DataLoader(train_dataset, batch_size=16, shuffle=True)
        test_loader = DataLoader(test_dataset, batch_size=16, shuffle=False)

        criterion = nn.CrossEntropyLoss()
        optimizer = torch.optim.Adam(self.model.parameters(), lr=0.0002)

        # Create a tqdm object
        progress_bar = tqdm(range(15), desc="Training", leave=False)

        for epoch in progress_bar:
            total_loss = 0
            for X_batch, y_batch in train_loader:
                optimizer.zero_grad()
                outputs = self.model(X_batch)
                loss = criterion(outputs, torch.max(y_batch, 1)[1])
                loss.backward()
                optimizer.step()
                total_loss += loss.item()

            # Calculate accuracy on the validation set
            val_acc = 0
            with torch.no_grad():
                for X_val, y_val in test_loader:
                    outputs = self.model(X_val)
                    _, predicted = torch.max(outputs, 1)
                    val_acc += (predicted == torch.max(y_val, 1)[1]).sum().item()
            accuracy = val_acc / len(y_val_tensor)

            # If the current model has better accuracy, save the model parameters
            if accuracy > self.best_accuracy:
                self.best_accuracy = accuracy
                self.best_model_params = self.model.state_dict()

            # Update the progress bar
            progress_bar.set_postfix({'Loss': f'{total_loss:.4f}', 'Accuracy': f'{self.best_accuracy:.4f}'})
            
        # Save the best model parameters to the model
        self.save_best_model('best_model.pth')

    def predict(self, X):
        X_tensor = torch.FloatTensor(X)
        with torch.no_grad():
            # Set the best_model_params to the model
            predictions = self.model(X_tensor)
        _, predicted = torch.max(predictions, 1)
        return predicted.numpy()

    def save_best_model(self, filepath):
        torch.save(self.best_model_params, filepath)


In [31]:
# Initialize the PyTorchClassifier
pytorch_clf = PyTorchClassifier(X_train_transformed.shape[1], 512, 512, len(labels) , learning_rate=0.0002, epoch=15)

# Fit the PyTorchClassifier
# pytorch_clf.fit(X_train_transformed, X_test_transformed, y_train, y_test)
pytorch_clf.fit(X_train_transformed, X_test_transformed, y_train, y_test)

# Predict the labels of the test set
y_pred = pytorch_clf.predict(X_test_transformed)

# Predict the labels of the training set
y_pred_train = pytorch_clf.predict(X_train_transformed)

# Print the accuracy of the Random Forest Classifier model on the training set
print(f"Accuracy of Neural Network Classifier model on the training set: {accuracy_score(y_train, y_pred_train)*100}")

# Print the accuracy of the Random Forest Classifier model on the test set
print(f"Accuracy of Neural Network Classifier model: {accuracy_score(y_test, y_pred)*100}")

# Print the classification report of the Random Forest Classifier model
print(classification_report(y_train, y_pred_train, target_names=labels))

# Print the classification report of the Random Forest Classifier model
print(classification_report(y_test, y_pred, target_names=labels))


Accuracy of Neural Network Classifier model on the training set: 100.0
Accuracy of Neural Network Classifier model: 96.83333333333334
                      precision    recall  f1-score   support

    Scheherazade New       1.00      1.00      1.00       850
              Marhey       1.00      1.00      1.00       850
            Lemonada       1.00      1.00      1.00       847
IBM Plex Sans Arabic       1.00      1.00      1.00       850

            accuracy                           1.00      3397
           macro avg       1.00      1.00      1.00      3397
        weighted avg       1.00      1.00      1.00      3397

                      precision    recall  f1-score   support

    Scheherazade New       0.96      0.95      0.96       150
              Marhey       0.95      0.96      0.96       150
            Lemonada       0.99      1.00      1.00       150
IBM Plex Sans Arabic       0.97      0.96      0.96       150

            accuracy                           0.97    

In [147]:
# Build a neural networ consist of input layer of size 400, hidden layer of size 256 , hidden layer of size 128 and output layer of size 4 pytorch


# Convert the data to tensors
X_train_tensor = torch.tensor(X_train_transformed, dtype=torch.float32)
y_train_tensor = torch.tensor(y_train, dtype=torch.long)
X_test_tensor = torch.tensor(X_test_transformed, dtype=torch.float32)
y_test_tensor = torch.tensor(y_test, dtype=torch.long)

# Print the shape of the tensors
print(X_train_tensor.shape, y_train_tensor.shape, X_test_tensor.shape, y_test_tensor.shape)

# Create a dataset
train_dataset = TensorDataset(X_train_tensor, y_train_tensor)
test_dataset = TensorDataset(X_test_tensor, y_test_tensor)

# Create a dataloader
train_loader = DataLoader(train_dataset, batch_size=16, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=16, shuffle=False)

# Define the neural network
class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        self.fc1 = nn.Linear(X_train_tensor.shape[1], 512)
        self.fc2 = nn.Linear(512, 256)
        self.fc3 = nn.Linear(256, 4)
        
    def forward(self, x):
        x = torch.relu(self.fc1(x))
        x = torch.relu(self.fc2(x))
        x = self.fc3(x)
        return x
    
# Create the model
model = Net()

# Define the loss function
criterion = nn.CrossEntropyLoss()

# Define the optimizer
optimizer = optim.Adam(model.parameters(), lr=0.0002)

best_score = 0.0

# Train the model
for epoch in range(30):
    model.train()
    running_loss = 0.0
    pbar = tqdm(enumerate(train_loader), total=len(train_loader))
    for i, (X, y) in pbar:
        optimizer.zero_grad()
        output = model(X)
        loss = criterion(output, y)
        loss.backward()
        optimizer.step()

        # update running loss value
        running_loss += loss.item()

        # update the progress bar with the average loss
        pbar.set_description(f"Epoch {epoch+1} Loss: {running_loss/(i+1):.4f}")

    # reset running loss for next epoch
    running_loss = 0.0

    # Evaluate the model
    model.eval()

    # Get the predictions
    y_pred = []
    y_true = []
    for X, y in test_loader:
        output = model(X)
        y_pred.extend(torch.argmax(output, 1).tolist())
        y_true.extend(y.tolist())

    # Calculate the accuracy
    accuracy = accuracy_score(y_true, y_pred)*100
    print('Accuracy: ', accuracy)

    # If the accuracy is better than the best score so far, save the model
    if accuracy > best_score:
        best_score = accuracy
        torch.save(model.state_dict(), 'best_model.pth')

# Print the classification report
print(classification_report(y_true, y_pred, target_names=labels))
# Accuracy:  96.75%

torch.Size([3397, 3165]) torch.Size([3397]) torch.Size([600, 3165]) torch.Size([600])


Epoch 1 Loss: 0.6915: 100%|██████████| 213/213 [00:02<00:00, 90.17it/s]


Accuracy:  95.83333333333334


Epoch 2 Loss: 0.0215: 100%|██████████| 213/213 [00:02<00:00, 90.87it/s]


Accuracy:  96.16666666666667


...

Accuracy:  97.0


Epoch 29 Loss: 0.0000: 100%|██████████| 213/213 [00:02<00:00, 90.71it/s]


Accuracy:  97.0


Epoch 30 Loss: 0.0000: 100%|██████████| 213/213 [00:02<00:00, 93.29it/s]


Accuracy:  97.0
                      precision    recall  f1-score   support

    Scheherazade New       0.96      0.97      0.96       150
              Marhey       0.94      0.97      0.95       150
            Lemonada       0.99      1.00      1.00       150
IBM Plex Sans Arabic       0.99      0.95      0.97       150

            accuracy                           0.97       600
           macro avg       0.97      0.97      0.97       600
        weighted avg       0.97      0.97      0.97       600



In [176]:
# Stacking the models together to get the best model
from sklearn.ensemble import StackingClassifier

# Define the base models
# Stacking the models together to get the best model

# Define the base models
estimators = [
    # ('logistic', LogisticRegression(warm_start=True, solver='saga', penalty='l2', C=0.8, random_state=42)),
    ('neural_net1', PyTorchClassifier(X_train_transformed.shape[1], 512, 512, len(labels) , learning_rate=0.0002, epoch=10)),
    ('logistic',  LogisticRegression(warm_start=True, solver='saga', penalty='l2', C=0.8, random_state=42))
    
]
meta_model = LogisticRegression()

# Initialize the stacking classifier
stacking_clf = StackingClassifier(estimators=estimators, final_estimator=meta_model, cv=5)

# Fit the stacking classifier
stacking_clf.fit(X_train_transformed , y_train)

# Predict the labels of the test set
y_pred = stacking_clf.predict(X_test_transformed)

# Predict the labels of the training set
y_pred_train = stacking_clf.predict(X_train_transformed)

# Print the accuracy of the Stacking Classifier model on the training set
print(f"Accuracy of Stacking Classifier model on the training set: {accuracy_score(y_train, y_pred_train)*100}")

# Print the accuracy of the Stacking Classifier model on the test set
print(f"Accuracy of Stacking Classifier model: {accuracy_score(y_test, y_pred)*100}")

# Print the classification report of the Stacking Classifier model
print(classification_report(y_train, y_pred_train, target_names=labels))

# Print the classification report of the Stacking Classifier model
print(classification_report(y_test, y_pred, target_names=labels))

# ACCURACY: 96%

C:\Users\ahmed\AppData\Roaming\Python\Python311\site-packages\sklearn\linear_model\_sag.py:350: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn(
C:\Users\ahmed\AppData\Roaming\Python\Python311\site-packages\sklearn\linear_model\_sag.py:350: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn(
C:\Users\ahmed\AppData\Roaming\Python\Python311\site-packages\sklearn\linear_model\_sag.py:350: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn(
C:\Users\ahmed\AppData\Roaming\Python\Python311\site-packages\sklearn\linear_model\_sag.py:350: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn(
C:\Users\ahmed\AppData\Roaming\Python\Python311\site-packages\sklearn\linear_model\_sag.py:350: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn(


Accuracy of Stacking Classifier model on the training set: 99.97056226081837
Accuracy of Stacking Classifier model: 96.0
                      precision    recall  f1-score   support

    Scheherazade New       1.00      1.00      1.00       850
              Marhey       1.00      1.00      1.00       850
            Lemonada       1.00      1.00      1.00       847
IBM Plex Sans Arabic       1.00      1.00      1.00       850

            accuracy                           1.00      3397
           macro avg       1.00      1.00      1.00      3397
        weighted avg       1.00      1.00      1.00      3397

                      precision    recall  f1-score   support

    Scheherazade New       0.97      0.95      0.96       150
              Marhey       0.95      0.93      0.94       150
            Lemonada       0.99      0.99      0.99       150
IBM Plex Sans Arabic       0.93      0.97      0.95       150

            accuracy                           0.96       600
      

C:\Users\ahmed\AppData\Roaming\Python\Python311\site-packages\sklearn\linear_model\_sag.py:350: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn(
